## Syntax-dependent trace



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
CACHE_DIR = "/scratch/lucchetti.f/models/"
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR


import sys
sys.path.append('..')

import gc
import torch
from model_utils import *
import tqdm as notebook_tqdm
from model import ModelLoader

# from model_loader import ModelLoader
# from model_manager import ModelManager
# from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

#MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/santacoder"

check_devs()

/home/lucchetti.f/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 34079899648 used for device 0, reserved 0


In [ ]:
lm = ModelLoader(MODEL_NAME)

In [36]:
check_devs()

2357992960 / 34079899648 used for device 0, reserved 2373976064


## Inference

In [11]:
model = lm.model
tokenizer = lm.tokenizer

prompts = ["""# intialize three integers
           int x = 1
           int y = 2
           int z ="""]

prompts = ["""int x ="""]

# vanilla generate:

tokenized = tokenizer(prompts, return_tensors="pt")
outputs = model.generate(tokenized.input_ids.to(model.device), max_new_tokens=2, top_k = 1)
tokenizer.decode(outputs[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


'int x = 0'

## Trace + activation patch

In [90]:
txt, ret_dict = lm.generate(
    prompts,
    max_out_len=5,
    argmax_greedy= True,
    debug = True,
    request_activations= [lm.layer_name_format.format(i) for i in range(5,10)]
)
ret_dict

/home/lucchetti.f/codetrace/model.py:134: UserWarning: The model `bigcode/santacoder` of type `gpt2` already implements or can't utilize `use_cache` for fast generation. Setting `use_cache = False`.
  warnings.warn(f"The model `{type(self.model)}` can't utilize `use_cache` for fast generation. Setting `use_cache = False`.")


prompt <0> ==> [(' ', 0.6357), (' (', 0.0573), (' x', 0.0188), (' -', 0.0176), (' i', 0.0134)]
prompt <0> ==> [('0', 0.8682), ('1', 0.082), ('2', 0.0167), ('3', 0.0099), ('5', 0.0082)]


{'input_tokenized': [[('int', 389), (' x', 727), (' =', 256)]],
 'generated_tokens': [[[{'token': ' ', 'id': 207, 'p': 0.6357421875},
    {'token': ' (', 'id': 305, 'p': 0.05731201171875},
    {'token': ' x', 'id': 727, 'p': 0.0187530517578125},
    {'token': ' -', 'id': 459, 'p': 0.017608642578125},
    {'token': ' i', 'id': 583, 'p': 0.013397216796875}],
   [{'token': '0', 'id': 15, 'p': 0.8681640625},
    {'token': '1', 'id': 16, 'p': 0.08203125},
    {'token': '2', 'id': 17, 'p': 0.01666259765625},
    {'token': '3', 'id': 18, 'p': 0.00994873046875},
    {'token': '5', 'id': 20, 'p': 0.00824737548828125}]]],
 'activations': {'transformer.h.5': array([[[ 0.8833 , -0.8457 ,  0.2708 , ..., -0.595  , -0.51   ,
            0.502  ],
          [-0.2861 ,  0.1284 , -0.0094 , ...,  0.191  , -0.9004 ,
            0.496  ],
          [ 0.376  , -0.4688 ,  0.1257 , ...,  0.0729 ,  0.279  ,
            0.4692 ],
          [-0.1093 , -0.8286 ,  0.626  , ..., -0.05634,  0.3862 ,
           -0.09